In [1]:
#Same accross all Model Selection notebooks
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline

#Hyperparameter search method, metric: "roc_auc"
from sklearn.model_selection import GridSearchCV
#metric
from sklearn.metrics import roc_auc_score as auc

#Different for different Model Selection notebooks
#preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#model
from sklearn.neighbors import KNeighborsClassifier

In [2]:
#vvvvvvvvvvvvvv
model_name="KNeighborsClassifier"
#^^^^^^^^^^^^^^
#select the index of file to load


train_files=["DL_train.csv", #0
             "Morgan_train.csv", #1
             "Both_train.csv"] #2

test_files=["DL_test.csv" #0
            ,"Morgan_test.csv" #1
            ,"Both_test.csv"] #2

dataset_names=["DL", #0
               "Morgan", #1
               "Both"] #2

#vvvvvvvvvvvvvv
data_index=1
#^^^^^^^^^^^^^^

norm_type = [StandardScaler(), #0
             MinMaxScaler()]  #1
norm_names=["Standard-Scaler","Min-Max"]



#vvvvvvvvvvvvvv
norm_index=0
#^^^^^^^^^^^^^^

models=[
    KNeighborsClassifier,#0
    
]

model_calls=[m() for m in models]


#vvvvvvvvvvvvvv
model_index=0
#^^^^^^^^^^^^^^

#vvvvvvvvvvvvvv
param_grid = {
    "model__n_neighbors":[1,2]
}
#^^^^^^^^^^^^^^



# 1. Import the train and test datasets *

In [3]:


dataset_name=dataset_names[data_index]
norm_name=norm_names[norm_index]
#check dataset if there is a index col, id there is include the argument index_col=0
x_train = pd.read_csv(train_files[data_index],index_col=0)
y_train = x_train.ACTIVE
x_train.drop(["ACTIVE"],axis=1,inplace=True)
x_test = pd.read_csv(test_files[data_index],index_col=0) 

# 2. Setup Pipeline *

In [4]:
#Step 1: Normalise the data

#Step 2: Model fit


#Step 3: Pipeline
pipeline=Pipeline(steps=[("norm",norm_type[norm_index]),("model",model_calls[model_index])])

# 3. Setup Parameter Grid *

In [5]:
param_grid

{'model__n_neighbors': [1, 2]}

# 4. Setup GridSearch

In [6]:
score_metric = "roc_auc"
search = GridSearchCV(pipeline,param_grid,
                      n_jobs=-1,
                      cv=10,
                      scoring=score_metric,
                     )


TypeError: __init__() got an unexpected keyword argument 'random_state'

In [ ]:
search.fit(x_train,y_train)

In [ ]:
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

In [ ]:
search_res = pd.DataFrame(search.cv_results_)
search_res = search_res.sort_values(by=['rank_test_score'])
search_res.head()

# 5.Estimation of Model Performance based on CV in GridSearch

In [ ]:
estimated_auc = float(search_res.query("rank_test_score == 1").mean_test_score)

# 6.Best Hyperparameter *

In [ ]:
parms = search_res.query("rank_test_score == 1").params

In [ ]:
parms

In [ ]:
parms.values

In [ ]:
n_neighbors = parms.values[0]["model__n_neighbors"]
n_neighbors

# 7.Train model on full train dataset

In [ ]:
full_model = models[model_index](n_neighbors=n_neighbors)

In [ ]:
full_model.fit(x_train,y_train)

# 8.Prediction score for test dataset

In [ ]:
pred_test = pd.DataFrame(full_model.predict_proba(x_test))[1]


In [ ]:
pred_test.head()

# 9.Export result

In [ ]:
result = pd.DataFrame([estimated_auc,])

In [ ]:
result.head()

In [ ]:
result = pd.concat([result,pred_test])

In [ ]:
result.head()

In [ ]:
result.to_csv(f"{model_name}_{norm_name=n}_{dataset_name}_Result.csv")